# YOLOv11 OBB: TRAINING AND TEST

## 1) Instalation

In [ ]:
!pip -q install -U ultralytics
!pip install pandas
import ultralytics, sys
print('Ultralytics version:', ultralytics.__version__)
print('Python version:', sys.version)

## 2) Configuration

In [ ]:
from pathlib import Path
DATA_DIR = Path('/workspace/X_dataset/Sequential_Gap/OBB') 
DATA_YAML = DATA_DIR / 'data.yaml'
HYP_YAML = DATA_DIR / 'hyp2.yaml'

# Base OBB model (also s/m/l/x)
MODEL_NAME = 'yolo11n-obb.pt'

## 3) Training

In [ ]:
import yaml
from ultralytics import YOLO

# Load model (pretrained weights or custom architecture)
model = YOLO(MODEL_NAME)

# Load training hyperparameters from YAML file
with open(HYP_YAML, "r") as f:
    hyp = yaml.safe_load(f)

# Train the YOLO model
results = model.train(
    data=str(DATA_YAML),  # dataset config (paths, classes)
    imgsz=1536,           # input image size
    epochs=60,            # maximum training epochs
    batch=4,              # batch size per device
    device=DEVICE,        # GPU/CPU device (e.g. 0, "0,1", or "cpu")
    optimizer="AdamW",    # optimizer type
    close_mosaic=12,      # disable mosaic augmentation in last epochs
    patience=20,          # early stopping patience
    nbs=32,               # nominal batch size (for LR scaling)
    **hyp,                # additional hyperparameters from YAML
)

# Display training results
results


## 4) Test

In [ ]:
from ultralytics import YOLO
import os
import pandas as pd

# Model and dataset paths
model_path = "YOLOV11N-C.pt"
DATA_YAML = "/workspace/C_dataset/Sequential_Gap/OBB/data.yaml"
SAVE_DIR = "resultados_test_aabb"
MODEL_NAME = "modelo_AABB_CC"

# Load trained model
model = YOLO(model_path)

# Run evaluation on test split
metrics_test = model.val(
    data=DATA_YAML,
    split='test',       # evaluate on test set
    imgsz=1024,         # image size for evaluation
    project=SAVE_DIR,   # root directory for saving results
    name=MODEL_NAME,    # subdirectory name
    plots=True,         # save plots (PR curves, confusion matrix, etc.)
    save_json=True      # export COCO-style JSON metrics
)

# Extract global metrics
mp = metrics_test.box.mp         # mean precision
mr = metrics_test.box.mr         # mean recall
map50 = metrics_test.box.map50   # mAP at IoU=0.5
map5095 = metrics_test.box.map   # mAP at IoU=0.5:0.95

# Collect results in a list of dicts
resultados = []
resultados.append({
    "Model": MODEL_NAME,
    "Class": "GLOBAL",
    "Precision": mp,
    "Recall": mr,
    "AP@0.5": map50,
    "AP@0.5:0.95": map5095
})

# Per-class metrics
names = model.names
for i, name in names.items():
    p, r, ap50, ap = metrics_test.box.class_result(i)
    resultados.append({
        "Model": MODEL_NAME,
        "Class": name,
        "Precision": p,
        "Recall": r,
        "AP@0.5": ap50,
        "AP@0.5:0.95": ap
    })

# Save metrics to CSV in the same directory as YOLO results
output_dir = metrics_test.save_dir
csv_path = os.path.join(output_dir, f"{MODEL_NAME}_metrics.csv")
pd.DataFrame(resultados).to_csv(csv_path, index=False)

print(f"Metrics saved at: {csv_path}")


## 5) Predictions for Visualization

In [ ]:
#TIS SECTION IS USED TO SAVE THE PREDICTIONS IN A .txt FILE, FOR A BETTER VISUALIZATION THAT THE ONE PREDEFINED BY YOLO
from pathlib import Path
model_path = "YOLOV11N-B.pt"
model = YOLO(model_path)
SOURCE = '/workspace/C_dataset/Sequential_Gap/OBB/images/evaluation/v2_021960.jpg'
pred = model.predict(source=str(SOURCE), imgsz=1536, save=True, save_txt=True, save_conf=True)
pred